# AADT Confidence Interval - Interstate 99

## FHWA Links
* Guidelines for Obtaining AADT Estimates from Non-Traditional Sources:
    * https://www.fhwa.dot.gov/policyinformation/travel_monitoring/pubs/aadtnt/Guidelines_for_AADT_Estimates_Final.pdf
  
  
## AADT Analysis Locations
* 10 locations were used in the analysis
* Locations were determined based on the location on installed & recording Traffic Operations cameras
    * for additional information contact Zhenyu Zhu with Traffic Operations

## Traffic Census Data
* https://dot.ca.gov/programs/traffic-operations/census/traffic-volumes
* Back AADT, Peak Month, and Peak Hour usually represents traffic South or West of the count location.  
* Ahead AADT, Peak Month, and Peak Hour usually represents traffic North or East of the count location. Listing of routes with their designated  

* Because the Back & Ahead counts are included at each location in the Traffic Census Data, (e.g., "IRWINDALE, ARROW HIGHWAY") only one [OBJECTID*] per location was pulled; for this analysis the North Bound Nodes were used for the analysis. 
    * for more information see the diagram: https://traffic.onramp.dot.ca.gov/downloads/traffic/files/performance/census/Back_and_Ahead_Leg_Traffic_Count_Diagram.pdf

## StreetLight Analysis Data
* StreetLight Locations on Interstate 99 are one-direction, each location will contain two points: northbound and southbound
    * Analysis Type == Network Performance
    * Segment Metrics
    * 2022 was used to match currently available Traffic Census Data (as of 8/27/2025)
    * pulled a variety of Day Types, but plan to just look at """All Day Types"""
    * pulled a variety of Day Parts, but plan to just look at """All Day Parts"""




In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

### Pull in the Location Dictionaries

In [2]:
# pull in the coordinates from the utils docs
#from osow_frp_o_d_utils_v3 import origin_intersections, destination_intersections
from sr99_tc_locations_utils import sr_99_d3_tc_aadt_locations

### Identify the Google Cloud Storage path

In [3]:
# Identify the GCS path to the data
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/compare_traffic_counts/sr99_d3/"

### Pull in the Data

In [4]:
# This function will pull in the data and clean the column headers in a way that will make them easier to work with
def getdata_and_cleanheaders(path):
    # Read the CSV file
    df = pd.read_csv(path)

    # Clean column headers: remove spaces, convert to lowercase, and strip trailing asterisks
    cleaned_columns = []
    for column in df.columns:
        cleaned_column = column.replace(" ", "").lower().rstrip("*")
        cleaned_columns.append(cleaned_column)

    df.columns = cleaned_columns
    return df

In [5]:
# pull in the data & create dataframes
df_tc = getdata_and_cleanheaders(f"{gcs_path}caltrans_traffic_census_2022.csv")  # Traffic Census
df_stl = getdata_and_cleanheaders(f"{gcs_path}streetlight_sr99_d3_all_vehicles_2022_np.csv")  # StreetLight

In [6]:
# comparing
df_tc.to_csv("df_tc.csv", index=False)

In [7]:
# comparing
df_stl.to_csv("df_stl.csv", index=False)

In [8]:
def traditional_aadt_by_location(aadt_locations, df_tc, as_df=True):
    """
    Build a per-location summary of *traditional* (Traffic Census) AADT.

    Input:
      - aadt_locations: raw structure (dict or list) OR the normalized list/DF
        with columns/keys: name, daytype, objectids
      - df_tc: Traffic Census DataFrame (columns: objectid, ahead_aadt, back_aadt)
      - as_df: return a DataFrame (True) or list[dict] (False)

    Output columns:
      location, daytype, objectids, n_objectids, n_found_in_tc, missing_objectids,
      traditional_ahead_mean, traditional_behind_mean, traditional_aadt
    """

    # ----- tiny helpers kept INSIDE this function -----
    def _ensure_list(x):
        if x is None: return []
        if isinstance(x, (list, tuple, set)): return list(x)
        return [x]

    def _gather_objectids(node_dict):
        ids = []
        if not isinstance(node_dict, dict): return ids
        if "objectid" in node_dict:  ids.extend(_ensure_list(node_dict["objectid"]))
        if "objectids" in node_dict: ids.extend(_ensure_list(node_dict["objectids"]))
        return [str(i) for i in ids if i is not None and str(i).strip() != ""]

    def _dedup(seq):
        seen=set(); out=[]
        for x in seq:
            if x not in seen:
                out.append(x); seen.add(x)
        return out

    def _normalize_one_location(name, loc):
        nodes = loc.get("nodes", {}) or {}
        all_ids=[]
        for _, node in nodes.items():
            all_ids.extend(_gather_objectids(node))
        return {
            "name": name,
            "daytype": loc.get("daytype", "0: All Days (M-Su)"),
            "objectids": _dedup(all_ids),
        }

    def _normalize_input(aadt_locs):
        # Already normalized DataFrame?
        if isinstance(aadt_locs, pd.DataFrame) and \
           {"name","daytype","objectids"}.issubset(aadt_locs.columns):
            return aadt_locs.to_dict(orient="records")
        # Already normalized list[dict]?
        if isinstance(aadt_locs, list) and aadt_locs and isinstance(aadt_locs[0], dict) and \
           {"name","daytype","objectids"}.issubset(aadt_locs[0].keys()):
            return aadt_locs

        recs = []
        if isinstance(aadt_locs, dict):
            for nm, loc in aadt_locs.items():
                recs.append(_normalize_one_location(nm, loc))
        elif isinstance(aadt_locs, list):
            for item in aadt_locs:
                if isinstance(item, dict) and "nodes" in item:  # single location dict
                    nm = item.get("location_description") or item.get("name") or "UNKNOWN"
                    recs.append(_normalize_one_location(nm, item))
                elif isinstance(item, dict):  # dict keyed by name
                    for nm, loc in item.items():
                        recs.append(_normalize_one_location(nm, loc))
        return recs

    def _traditional_aadt_for_ids(df_tc_in, obj_ids):
        if not obj_ids:
            return np.nan, np.nan, np.nan, 0
        sub = df_tc_in[df_tc_in["objectid"].astype(str).isin(obj_ids)]
        if sub.empty:
            return np.nan, np.nan, np.nan, 0
        ahead_vals = pd.to_numeric(sub.get("ahead_aadt"), errors="coerce").dropna()
        back_vals  = pd.to_numeric(sub.get("back_aadt"),  errors="coerce").dropna()
        mean_ahead = ahead_vals.mean() if not ahead_vals.empty else np.nan
        mean_back  = back_vals.mean()  if not back_vals.empty  else np.nan
        overall = np.nanmean([mean_ahead, mean_back])  # average of the two means
        count_used = len(sub)
        return overall, mean_ahead, mean_back, count_used
    # ---------------------------------------------------

    norm = _normalize_input(aadt_locations)
    tc_ids_all = set(df_tc["objectid"].astype(str).unique())

    rows = []
    for loc in norm:
        obj_ids = [str(x) for x in (loc.get("objectids") or [])]
        overall, mean_ahead, mean_back, n_found = _traditional_aadt_for_ids(df_tc, obj_ids)
        missing = [x for x in obj_ids if x not in tc_ids_all]
        rows.append({
            "location": loc.get("name"),
            "daytype":  loc.get("daytype"),
            "objectids": ",".join(obj_ids),
            "n_objectids": len(obj_ids),
            "n_found_in_tc": int(n_found),
            "missing_objectids": ",".join(missing) if missing else "",
            "traditional_ahead_mean": mean_ahead,
            "traditional_behind_mean": mean_back,
            "traditional_aadt": overall,
        })

    return pd.DataFrame(rows) if as_df else rows

In [9]:
trad_df = traditional_aadt_by_location(sr_99_d3_tc_aadt_locations, df_tc, as_df=True)
trad_df.head()
trad_df.to_csv("traditional_aadt_by_location.csv", index=False)

In [10]:
def non_traditional_aadt_by_location(
    aadt_locations,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)",
    as_df=True
):
    """
    Build a per-location summary of *non-traditional* (StreetLight) AADT.

    Inputs:
      - aadt_locations: raw dict/list (your nested structure) OR normalized data
        with keys/cols: name, daytype, ahead_zones, behind_zones
      - df_stl: StreetLight DataFrame with columns:
          zonename, averagedailysegmenttraffic(stlvolume), daytype, daypart
        (and optionally modeoftravel if you want to filter by it)
      - daytype_filter, daypart_filter, modeoftravel_filter: filters to apply
      - zonename_col, stl_volume_col: column names in df_stl
      - as_df: return a DataFrame (True) or list[dict] (False)

    Output columns (one row per location):
      location, daytype_expected, daytype_used, daypart_used, modeoftravel_used,
      ahead_zones, behind_zones,
      non_trad_ahead_mean, non_trad_behind_mean, non_trad_aadt,
      stl_ahead_rows, stl_behind_rows, missing_ahead_zones, missing_behind_zones
    """

    # ------------- tiny helpers kept inside -------------
    def _ensure_list(x):
        if x is None: return []
        if isinstance(x, (list, tuple, set)): return list(x)
        return [x]

    def _gather_zones(node_dict):
        ahead  = _ensure_list(node_dict.get("zonename_ahead", []))
        behind = _ensure_list(node_dict.get("zonename_behind", []))
        return ahead, behind

    def _dedup(seq):
        seen=set(); out=[]
        for x in seq:
            if x not in seen:
                out.append(x); seen.add(x)
        return out

    def _normalize_one_location(name, loc):
        nodes = loc.get("nodes", {}) or {}
        ahead, behind = [], []
        for _, node in nodes.items():
            a, b = _gather_zones(node)
            ahead.extend([z for z in a if z])
            behind.extend([z for z in b if z])
        return {
            "name": name,
            "daytype": loc.get("daytype", "0: All Days (M-Su)"),
            "ahead_zones": _dedup(ahead),
            "behind_zones": _dedup(behind),
        }

    def _normalize_input(aadt_locs):
        # Already normalized DataFrame?
        if isinstance(aadt_locs, pd.DataFrame) and \
           {"name","daytype","ahead_zones","behind_zones"}.issubset(aadt_locs.columns):
            return aadt_locs.to_dict(orient="records")
        # Already normalized list[dict]?
        if isinstance(aadt_locs, list) and aadt_locs and isinstance(aadt_locs[0], dict) and \
           {"name","daytype","ahead_zones","behind_zones"}.issubset(aadt_locs[0].keys()):
            return aadt_locs

        # Otherwise, normalize from raw structure
        recs = []
        if isinstance(aadt_locs, dict):
            for nm, loc in aadt_locs.items():
                recs.append(_normalize_one_location(nm, loc))
        elif isinstance(aadt_locs, list):
            for item in aadt_locs:
                if isinstance(item, dict) and "nodes" in item:  # single location dict
                    nm = item.get("location_description") or item.get("name") or "UNKNOWN"
                    recs.append(_normalize_one_location(nm, item))
                elif isinstance(item, dict):  # dict keyed by name
                    for nm, loc in item.items():
                        recs.append(_normalize_one_location(nm, loc))
        return recs

    def _stl_means_for_zone_lists(stl_df, zones):
        """Return mean volume, matched row count, and missing list for a set of zonenames."""
        if not zones:
            return np.nan, 0, []
        sub = stl_df[stl_df[zonename_col].isin(zones)]
        present = set(sub[zonename_col].unique())
        missing = [z for z in zones if z not in present]
        vals = pd.to_numeric(sub[stl_volume_col], errors="coerce").dropna()
        mean_val = vals.mean() if not vals.empty else np.nan
        return mean_val, int(len(sub)), missing
    # ----------------------------------------------------

    # Filter StreetLight once
    filt = (df_stl["daytype"] == daytype_filter) & (df_stl["daypart"] == daypart_filter)
    if modeoftravel_filter and ("modeoftravel" in df_stl.columns):
        filt = filt & (df_stl["modeoftravel"] == modeoftravel_filter)
    stl_filtered = df_stl.loc[filt].copy()

    norm = _normalize_input(aadt_locations)

    rows = []
    for loc in norm:
        ahead = _ensure_list(loc.get("ahead_zones", []))
        behind = _ensure_list(loc.get("behind_zones", []))

        mean_ahead, ahead_n, miss_a = _stl_means_for_zone_lists(stl_filtered, ahead)
        mean_behind, behind_n, miss_b = _stl_means_for_zone_lists(stl_filtered, behind)
        overall = np.nanmean([mean_ahead, mean_behind])

        rows.append({
            "location": loc.get("name"),
            "daytype_expected": loc.get("daytype"),
            "daytype_used": daytype_filter,
            "daypart_used": daypart_filter,
            "modeoftravel_used": modeoftravel_filter if modeoftravel_filter else "",
            "ahead_zones": ",".join(ahead),
            "behind_zones": ",".join(behind),

            "non_trad_ahead_mean": mean_ahead,
            "non_trad_behind_mean": mean_behind,
            "non_trad_aadt": overall,

            "stl_ahead_rows": ahead_n,
            "stl_behind_rows": behind_n,
            "missing_ahead_zones": ",".join(miss_a) if miss_a else "",
            "missing_behind_zones": ",".join(miss_b) if miss_b else "",
        })

    return pd.DataFrame(rows) if as_df else rows

In [11]:
# If you have the raw nested structure:
stl_df = non_traditional_aadt_by_location(
    sr_99_d3_tc_aadt_locations,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter="All Vehicles - StL All Vehicles Volume",  # or None
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)",
    as_df=True
)

stl_df.to_csv("non_traditional_aadt_by_location.csv", index=False)

In [12]:
# ------------------------------------------------------
# 4) Build the per-location comparison DataFrame
# ------------------------------------------------------
def build_aadt_comparison_df(
    aadt_locations,
    df_tc,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)"
):
    """
    Returns a tidy DataFrame with one row per (location), including:
      • objectids, ahead_zones, behind_zones
      • traditional_aadt, traditional_ahead_mean, traditional_behind_mean
      • non_trad_aadt, non_trad_ahead_mean, non_trad_behind_mean
      • TCE (%)
      • counts & missing info for debugging
    """
    records = []

    for loc in _iter_locations(aadt_locations):
        # traditional
        trad_overall, trad_ahead, trad_behind, n_ids = _traditional_aadt_for_ids(
            df_tc, loc["objectids"]
        )

        # non-traditional
        stl_overall, stl_ahead, stl_behind, ahead_n, behind_n, miss_a, miss_b = _stl_aadt_for_zones(
            df_stl,
            loc["ahead_zones"],
            loc["behind_zones"],
            daytype=daytype_filter,
            daypart=daypart_filter,
            modeoftravel=modeoftravel_filter,
            zonename_col=zonename_col,
            stl_volume_col=stl_volume_col
        )

        # TCE
        tce = np.nan
        if pd.notna(trad_overall) and trad_overall != 0 and pd.notna(stl_overall):
            tce = 100.0 * (stl_overall - trad_overall) / trad_overall

        records.append({
            "location": loc["name"],
            "daytype_expected": loc["daytype"],
            "daytype_used": daytype_filter,
            "daypart_used": daypart_filter,
            "objectids": ",".join(loc["objectids"]),
            "n_objectids": len(loc["objectids"]),
            "ahead_zones": ",".join(loc["ahead_zones"]),
            "behind_zones": ",".join(loc["behind_zones"]),

            "traditional_aadt": trad_overall,
            "traditional_ahead_mean": trad_ahead,
            "traditional_behind_mean": trad_behind,

            "non_trad_aadt": stl_overall,
            "non_trad_ahead_mean": stl_ahead,
            "non_trad_behind_mean": stl_behind,

            "tce_percent": tce,

            "stl_ahead_rows": ahead_n,
            "stl_behind_rows": behind_n,
            "missing_ahead_zones": ",".join(miss_a) if miss_a else "",
            "missing_behind_zones": ",".join(miss_b) if miss_b else "",
        })

    df = pd.DataFrame.from_records(records)
    # Optional: keep a stable, readable column order
    preferred_cols = [
        "location", "objectids", "n_objectids",
        "ahead_zones", "behind_zones",
        "traditional_ahead_mean", "traditional_behind_mean", "traditional_aadt",
        "non_trad_ahead_mean", "non_trad_behind_mean", "non_trad_aadt",
        "tce_percent",
        "daytype_expected", "daytype_used", "daypart_used",
        "stl_ahead_rows", "stl_behind_rows",
        "missing_ahead_zones", "missing_behind_zones",
    ]
    df = df[[c for c in preferred_cols if c in df.columns]]
    return df


In [13]:
# ------------------------------------------------------
# 5) Confidence interval over TCE
# ------------------------------------------------------
def tce_confidence_interval(detail_df, confidence=0.95):
    """
    Computes mean TCE, CI, t-critical, and t-test statistic over the rows in detail_df.
    Expects a 'tce_percent' column.
    """
    tces = pd.to_numeric(detail_df["tce_percent"], errors="coerce").dropna().values
    n = len(tces)
    if n == 0:
        return None, None, None, None, None

    mean_tce = float(np.mean(tces))
    std_tce = float(np.std(tces, ddof=1)) if n > 1 else 0.0
    se = std_tce / np.sqrt(n) if n > 1 else 0.0

    if n > 1 and se > 0:
        dfree = n - 1
        tcrit = float(stats.t.ppf((1 + confidence) / 2, dfree))
        ci_lo = mean_tce - tcrit * se
        ci_hi = mean_tce + tcrit * se
        t_stat = mean_tce / se
    else:
        tcrit = None
        ci_lo = None
        ci_hi = None
        t_stat = None

    return mean_tce, ci_lo, ci_hi, tcrit, t_stat

In [14]:
# 1) Build the per-location comparison table
detail = build_aadt_comparison_df(
    aadt_locations=sr_99_d3_tc_aadt_locations,
    df_tc=df_tc,
    df_stl=df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter="All Vehicles - StL All Vehicles Volume",  # or None if not needed
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)"
)

# e.g., export for inspection
# detail.to_csv("sr99_d3_aadt_comparison.csv", index=False)

# 2) Compute the CI across locations
mean_tce, ci_lo, ci_hi, tcrit, t_stat = tce_confidence_interval(detail, confidence=0.95)
print(mean_tce, ci_lo, ci_hi, tcrit, t_stat)

NameError: name '_iter_locations' is not defined

### Mean TCE: -3.62
Traffic Census Error (TCE)
* 

### 95% Confidence Interval (-10.78%, 3.54%)
* 

### T-Test Statistic  
* 
### Summary
* 
